# Формирование учебного датасета к проекту SkiilFactory
## «Кто хочет стать миллионером кинопроката?»
### На основе открытых данных IMDb

В старом учебном датасете к проекту всего 1889 фильмов. Из-за этого с датасетом работать скучновато.\
А что, если увеличить размер датасета на пару порядков? Стало бы гораздо интереснее.

Надо всего лишь загрузить данные с официального сайта IMDb и перевести датасет в нужный формат.

In [ ]:
import pandas as pd
import io, os, gzip, shutil

from urllib.request import urlopen

# Официальное расположение датасетов IMDB
PATH_IMDB_DATASETS = 'https://datasets.imdbws.com/'
DATASET_MOVIES = 'title.basics.tsv.gz' # фильмы
DATASET_RATINGS = 'title.ratings.tsv.gz' # оценки
DATASET_CASTS = 'title.principals.tsv.gz' # сведения о съемочных группах, в т.ч. id актеров и режиссеров
DATASET_PEOPLES = 'name.basics.tsv.gz' # информация о людях по их id

![IMDB logo](https://m.media-amazon.com/images/G/01/IMDb/BG_rectangle._CB1509060989_SY230_SX307_AL_.png)
# [IMDb Datasets](https://www.imdb.com/interfaces/)
Subsets of IMDb data are available for access to customers for personal and non-commercial use. You can hold local copies of this data, and it is subject to our terms and conditions. Please refer to the Non-Commercial Licensing and copyright/license and verify compliance.

## Data Location

The dataset files can be accessed and downloaded from https://datasets.imdbws.com/. The data is refreshed daily. ***(НЕТ, увы...)***

## IMDb Dataset Details

Each dataset is contained in a gzipped, tab-separated-values (TSV) formatted file in the UTF-8 character set. The first line in each file contains headers that describe what is in each column. A ‘`\N`’ is used to denote that a particular field is missing or null for that title/name. The available datasets are as follows:

### title.basics.tsv.gz - Contains the following information for titles:
- `tconst` (string) - alphanumeric unique identifier of the title
- `titleType` (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
- `primaryTitle` (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
- `originalTitle` (string) - original title, in the original language
- `isAdult` (boolean) - 0: non-adult title; 1: adult title
- `startYear` (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
- `endYear` (YYYY) – TV Series end year. ‘`\N`’ for all other title types
- `runtimeMinutes` – primary runtime of the title, in minutes
- `genres` (string array) – includes up to three genres associated with the title
### title.ratings.tsv.gz – Contains the IMDb rating and votes information for titles
- `tconst` (string) - alphanumeric unique identifier of the title
- `averageRating` – weighted average of all the individual user ratings
- `numVotes` - number of votes the title has received
### title.principals.tsv.gz – Contains the principal cast/crew for titles
- `tconst` (string) - alphanumeric unique identifier of the title
- `ordering` (integer) – a number to uniquely identify rows for a given titleId
- `nconst` (string) - alphanumeric unique identifier of the name/person
- `category` (string) - the category of job that person was in
- `job (string)` - the specific job title if applicable, else ‘`\N`’
- `characters` (string) - the name of the character played if applicable, else ‘`\N`’
### name.basics.tsv.gz – Contains the following information for names:
- `nconst` (string) - alphanumeric unique identifier of the name/person
- `primaryName` (string)– name by which the person is most often credited
- `birthYear` – in YYYY format
- `deathYear` – in YYYY format if applicable, else ‘`\N`’
- `primaryProfession` (array of strings)– the top-3 professions of the person
- `knownForTitles` (array of tconsts) – titles the person is known for

In [ ]:
# Не будем скачивать новую базу, если датасеты лежат в подпапке /datasets. Создадим папку, если ее нет
# Для обновления датасетов надо просто удалить файлы из этой папки или папку целиком
os.makedirs('datasets', exist_ok=True)

In [ ]:
# Загрузка любого датасета IMDB производится одним и тем же способом
# Поэтому создадим функцию для загрузки датасета
def load_IMDB_dataset(dataset_name, return_dataframe = True):
    # Проверяем, есть ли у нас уже скачанный датасет
    if not dataset_name in os.listdir('datasets'):
        # Загружаем датасет с сайта IMDB
        print(f'Скачиваем `{dataset_name}`')
        with urlopen(PATH_IMDB_DATASETS + dataset_name) as f_imdb_site:
            with open('datasets/' + dataset_name, 'wb') as f_local:
                f_local.write(f_imdb_site.read())
    else:
        print(f'Обнаружен датасет `{dataset_name}`')

    # Вычислим и покажем размер файла с датасетом в байтах
    print(f'Размер датасета `{dataset_name}` составляет {os.path.getsize("datasets/" + dataset_name)} байт')

    if return_dataframe:
        # Загрузим датасет из скачанного файла. Разделитель \t, неопределенные значения записаны как \N, файл сжат gzip
        # dtype=str нужен, чтобы быстрее загрузить датасет, иначе python анализирует каждую ячейку. Пусть грузит строки
        return pd.read_csv("datasets/" + dataset_name, sep='\t', dtype=str, na_values= r'\N', compression='gzip')
    else:
        return None

In [ ]:
# Этот блок кода нужен при повторном запуске Jupiter-блокнота в ходе одного сеанса
# Оказалось, что при повторной загрузке в память нужно удалять датафрейм явным способом при помощи del
# Самый простой способ узнать, была ли переменная ранее определена, поискать ее в vars()
if 'data_source' in vars():
    del data_source

In [ ]:
%%time
# Загрузим информацию о фильмах
data_source = load_IMDB_dataset(DATASET_MOVIES)

In [ ]:
%%time
# Из описания датасета мы знаем, что titleType содержит тип записи. Посмотрим внимательнее на содержимое этого поля
data_source.titleType.value_counts()

In [ ]:
%%time
# Можно изучить оригинальный датасет
data_source.info(null_counts=True) # без counts=True не будет выведена информация о Non-Null
# data_source.describe(include='all') # можно было использовать include='object', но так надежнее

In [ ]:
# Посмотрим на запись о фильмах с пустыми полями startYear, runtimeMinutes или genres
data_source[(data_source.titleType == 'movie') 
    &(   data_source.startYear.isna()
        |data_source.runtimeMinutes.isna()
        |data_source.genres.isna()
    )]

In [ ]:
# Скорее всего, речь идет об утерянных фильмах или невышедших, но аносированных премьерах
# Не будем брать их в нашу выборку 

In [ ]:
%%time
# Сформируем нужную нам выборку фильмов (titleType == 'movie').
# Если для фильма неизвестен год выпуска, его длительность или жанр, то их мы не берем. Зачем нам такое?
# в последних строках выбираем столбцы, которые хотим оставить и переименовываем их
data_movies = data_source[(data_source.titleType == 'movie')
    # Чтобы оставить в датасете NaN (например, в учебных челях), достаточно удалить или закомментировать фильтпры
    &data_source.startYear.notna()
    &data_source.runtimeMinutes.notna()
    &data_source.genres.notna()
    ] \
    [['tconst', 'originalTitle', 'startYear', 'runtimeMinutes', 'genres']] \
    .set_axis(['imdb_id', 'original_title', 'release_year', 'runtime', 'genres'], axis = 'columns')

# заменим заяпяые на `|` в списке жанров
data_movies.genres = data_movies.genres.str.replace(',','|')

In [ ]:
data_movies

In [ ]:
%%time
# Загрузим информацию об оценках
data_source = load_IMDB_dataset(DATASET_RATINGS)

In [ ]:
%%time
# Изучим датасет оценок
data_source.info(null_counts=True)
data_source.describe(include='all')

In [ ]:
%%time
# Оставим информацию о рейтингах фильмов из нашей выборки
# И переименуем столбец, содержащий imdb_id
data_ratings = data_source[(data_source.tconst.isin(data_movies.imdb_id))] \
    .rename(columns={'tconst': 'imdb_id'})

# Сразу надо освободить память от неиспользуемых объектов
if 'data_source' in vars():
    del data_source

# Посмотрим на датасет
data_ratings.info()
data_ratings.describe(include='all')

In [ ]:
# Сразу надо освободить память от неиспользуемых объектов
if 'data_source' in vars():
    del data_source

# Посмотрим на датасет
data_movies.info()
data_movies.describe(include='all')

In [ ]:
%%time

# Попродуем загрузить датасет с информацией о съемочных группах с опцией LOW_MEMORY_OPTION = False
LOW_MEMORY_OPTION = True

if not LOW_MEMORY_OPTION:
    # Загрузим информацию о съемочных группах из полного датасета
    data_source = load_IMDB_dataset(DATASET_CASTS)

# Если датасет не загружается из-за нехватки памяти, то установим LOW_MEMORY_OPTION = True

In [ ]:
%%time

if LOW_MEMORY_OPTION:
    # Можно столкнуться с проблемами при загрузке полного датасета из-за нехватки памяти
    # В этом случае надо провести предварительную обработку обработку исходных данных

    # Освободим память
    if 'data_source' in vars():
        del data_source

    # Загрузим датасет без считывания датафрейма
    load_IMDB_dataset(DATASET_CASTS, return_dataframe = False)
    
    # Построчно прочитаем файл и сохраним его отфильтрованный вариант
    # Так как файл большой и фильтруется относительно долго (несколько минут), то добавим интерактивности
    line_counts = 0 # сколько записей обработано
    new_line_counts = 0 # сколько записей мы записали в новый датасет

    filter_imdb_id = set(data_movies.imdb_id.values) # если не преобразовать в сет, проверка вхождения очень медленная
    filter_category = set(['actor', 'actress', 'director']) # переход к сету при проверке категорий ускорил все в 4 раза

    # Так как датасет заархивирован, читаем прямо из архива. Распакованный файл весил бы около 2 гб
    with gzip.open('datasets/' + DATASET_CASTS, 'rt', encoding='utf-8') as f_original, \
        gzip.open('datasets/' + 'filtered_' + DATASET_CASTS, 'wt', encoding='utf-8') as f_filtered:
        for line_tsv in f_original:
            line_counts += 1
            tconst, ordering, nconst, category, job, characters = line_tsv.split('\t')
            if line_counts == 1:
                f_filtered.write(line_tsv) # записываем заголовок
            elif (category in filter_category) and (tconst in filter_imdb_id):
                f_filtered.write(line_tsv)
                new_line_counts += 1
            if line_counts % 77711 == 0: # любое число-интервал для обновления счетчика, взяли красивое простое число
                print(f'Обработано {line_counts} строк. Записано {new_line_counts} строк.     ', end='\r')
        else:
            print(f'Обработано {line_counts} строк. Записано {new_line_counts} строк.     ')

In [ ]:
%%time

if LOW_MEMORY_OPTION:
    # Загрузим информацию об актерах из отфильтрованнойго датасета
    data_source = load_IMDB_dataset('filtered_' + DATASET_CASTS)

In [ ]:
%%time
# Изучим датасет по съемочным группам
data_source.info(null_counts=True)
data_source.describe(include='all')

In [ ]:
%%time
# Интересно посмотреть, какие есть категории. В полном датасете, с опцией LOW_MEMORY_OPTION = False их больше
data_source[data_source.tconst.isin(data_movies.imdb_id)].category.value_counts()

In [ ]:
%%time
# Оставим информацию только об актерах и режиссерах только по фильмам из нашей выборки
# И переименуем столбец, содержащий imdb_id
data_casts = data_source[(data_source.tconst.isin(data_movies.imdb_id))
    &data_source.category.isin(['actor', 'actress', 'director'])] \
    .rename(columns={'tconst': 'imdb_id', 'nconst': 'people_id'}) # удобная функция для переименования пары колонок

In [ ]:
# Освободим память
if 'data_source' in vars():
    del data_source

# Посмотрим на датасет
data_casts.info(null_counts=True)

In [ ]:
# переименуем 'actor' и 'actress' в 'cast'
data_casts.category[(data_casts.category == 'actor') | (data_casts.category == 'actress')] = 'cast'
data_casts.describe(include='all')

In [ ]:
%%time
# Интересно, что в job?
data_casts.groupby(by = 'category').job.value_counts()

In [ ]:
%%time
# Загрузим информацию о людях
data_source = load_IMDB_dataset(DATASET_PEOPLES)

In [ ]:
%%time
# Изучим
data_source.info(null_counts=True)
# data_source.describe(include='all')

In [ ]:
%%time
# Оставим информацию о людях только из нашей выборки
data_peoples = data_source[['nconst', 'primaryName', 'birthYear']] \
    [(data_source.nconst.isin(data_casts.people_id))] \
    .rename(columns={'nconst': 'people_id'}) # и переименуем имя колонки с id

# Сразу надо освободить память от неиспользуемых объектов
# if 'data_source' in vars():
#     del data_source

# Посмотрим на датасет
data_peoples.info()
data_peoples.describe(include='all')

In [ ]:
# Все доступные датасеты загружены
# Вспомним, где у нас что лежит и сформируем итоговый датасет
list(data_movies), list(data_ratings), list(data_casts), list(data_peoples)

In [ ]:
%%time
# Для сворачивания в одну ячейку сведений о режисерах и актерах подготовим специальный датафрейм
df_casts_list = data_casts[['imdb_id', 'people_id', 'category']]\
    .merge(right = data_peoples, on ='people_id') \
    .assign(ordering = data_casts.ordering.astype(int)) \
    .sort_values(by=['imdb_id', 'category', 'ordering']) \
    .set_index('imdb_id') \
    [['category', 'primaryName']]

In [ ]:
df_casts_list

In [ ]:
%%time
# Заполним словари режиссеров и актеров, в качестве ключа используем imdb_id
cast_dict={}
for imdb_id, primaryName in df_casts_list[df_casts_list.category == 'cast'].primaryName.iteritems():
    cast_dict.setdefault(imdb_id, '')
    cast_dict[imdb_id] += '|' + primaryName

director_dict={}
for imdb_id, primaryName in df_casts_list[df_casts_list.category == 'director'].primaryName.iteritems():
    director_dict.setdefault(imdb_id, '')
    director_dict[imdb_id] += '|' + primaryName

In [ ]:
%%time
# Сформируем таблицу с данными в том же формате, что и у учебной базы
# Информации о бюджете, кассовых сборах, студих, дате релиза, описании и ключевых славах
# в открытом датасете IMDb, к сожалению, нет
data = data_movies \
    .merge(right = data_ratings.rename(columns={'averageRating': 'vote_average', 'numVotes': 'num_votes'}),
#            how='left', # если хотим получить выборку с фильмами без рейтинга, раскомментируем строку
           on='imdb_id') \
    .merge(right = pd.DataFrame(pd.Series(cast_dict).str[1:], columns=['cast']), 
#            how='left', # если хотим получить выборку с фильмами без актеров, раскомментируем строку
           left_on='imdb_id', right_index = True) \
    .merge(right = pd.DataFrame(pd.Series(director_dict).str[1:], columns=['director']), 
#            how='left', # если хотим получить выборку с фильмами без режиссеров, раскомментируем строку
           left_on='imdb_id', right_index = True) \
    .reset_index(drop = True) \
    [['imdb_id', 'original_title', 'cast', 'director', 'runtime', 'genres', 'vote_average', 'num_votes', 'release_year']]

data

In [ ]:
# Сохраняем итоговый файл, который возможно будет использовать в учебном проекте
# Можно в архиированном виде, pd.read_csv() по расширению поймет, что надо разархивировать данные при чтении из файла
data.to_csv('movie_imdb_dataset.csv.gz', index=False, compression='gzip')

# Этот файл можно скоппировать в каталог с проектом «Кто хочет стать миллионером кинопроката?»
# и прорешать его не на 1889, а на 183835 фильмах
# data = pd.read_csv('movie_imdb_dataset.csv.gz')

In [ ]:
# В качестве бонуса для тех, кто досмотрел до конца - код функции вывода постера по imdb_id

In [ ]:
import re
from IPython.core.display import Image
from urllib.request import urlopen

# Указанный ниже SECRET_TMDB_API_KEY может перестать работать, тогда надо будет получить новый
# Инструкция здесь - https://www.themoviedb.org/documentation/api/
SECRET_TMDB_API_KEY = '74c77374f25ceb2688ab912ddab305f7'

def imdb_poster(imdb_id):
    url_movie_info = "https://api.themoviedb.org/3/find/" + imdb_id \
        + "?api_key=" + SECRET_TMDB_API_KEY + "&external_source=imdb_id"
    try:
        url_pict = 'http://image.tmdb.org/t/p/w300' \
            + re.search(r'poster_path":"([^"]*)', urlopen(url_movie_info).read().decode()).group(1)
        return Image(url_pict)
    except:
        return None

In [ ]:
# Выведем информацию и постер одного из фильмов
display(data[data.imdb_id=='tt1298650'])
imdb_poster('tt1298650')